# assertion

In [61]:
def waribiki(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price']

    return price


# 動く
product1 = {'name': 'ruby gem stone', 'price': 200000}
waribiki(product1, 0.3)

140000

In [64]:
# 壊れる
product1 = {'name': 'ruby gem stone', 'price': 200000}
waribiki(product1, 3)


AssertionError: 

assert は無効にできるのであんまり意味がない。デバック用として使うべし。

In [65]:
def waribiki(product, discount):
    price = int(product['price'] * (1.0 - discount))
    if discount > 1:
        raise ValueError('Discount should be less than 1.')
    if 0 <= price <= product['price']:
        raise ValueError('After-discount price should be less than before-discount price.')
    
    return price

# ちゃんとエラーが出る
product1 = {'name': 'ruby gem stone', 'price': 200000}
waribiki(product1, 2.0)

ValueError: Discount should be less than 1.

# try... finally & with

In [ ]:
# クローズするのがめんどい
file = open('file.txt', 'w')
try: 
    file.write('てすと')
finally:
    file.close()

In [25]:
# この方が楽
with open('file.txt', 'w') as file:
    file.write('テスト')

In [26]:
!pwd
!cat file.txt

/home/orz/python_note
テスト

## with(content manager)の仕組み

In [36]:
# enter - with に入った時に呼ばれる
# exit - 最後に呼ばれる

class FileManager:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print('entered')
        self.file = open(self.name, 'w')
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            print('closing...')
            self.file.close()

with FileManager('file2.txt') as file:
    file.write('てすと')
    file.write('終了')


entered
closing...
None
None
None


なぜ３つもargがいるのか？

> In order for __exit__ to work properly it must have exactly three arguments: exception_type, exception_value, and traceback. The formal argument names in the method definition do not need to correspond directly to these names, but they must appear in this order. If any exceptions occur while attempting to execute the block of code nested after the with statement, Python will pass information about the exception into the __exit__ method. You can then modify the definition of __exit__ to gracefully handle each type of exception.

In [32]:
!pwd
!cat file2.txt

/home/orz/python_note
てすと終了

## 用法

In [44]:
class IndentKun:
    def __init__(self):
        self.indent = 0

    def __enter__(self):
        self.indent += 1
        return self

    def print_context(self, context):
        print (('----' * self.indent) + context)

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level = 0


with IndentKun() as ind:
    ind.print_context('てすと')
    with ind:
        ind.print_context('テスト')
        with ind:
            ind.print_context('試験')

----てすと
--------テスト
------------試験


# var with Underscore

- _var 　
 - プライベート変数、プライベートメソッド

- var_  
    - 名前が予約語と衝突しないように使う
- _
    - temporaly varible
    - 一時的に使ったり、変数握りつぶして「どうでもいい」とつたえるため

## __var -> var for name mangling

In [60]:
class DoubleLeadingUnderScore:
    def __init__(self):
        self.test = 1
        self._test = 2
        self.__test = 3

    def  __test_method(self):
        print('mangled test method called')

test = DoubleLeadingUnderScore()
dir(test) # <= '_DoubleLeadingUnderScore__test'

['_DoubleLeadingUnderScore__test',
 '_DoubleLeadingUnderScore__test_method',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_test',
 'test']

In [56]:
print(test.test)
print(test._test)
print(test._DoubleLeadingUnderScore__test)

1
2
3


In [58]:
print(test.__test_method())

AttributeError: 'DoubleLeadingUnderScore' object has no attribute '__test_method'

In [62]:
test._DoubleLeadingUnderScore__test_method()

mangled test method called


おおっぴらに使うもんじゃない

>If your class is intended to be subclassed, and you have attributes that you do not want subclasses to use, consider naming them with double leading underscores and no trailing underscores. This invokes Python's name mangling algorithm, where the name of the class is mangled into the attribute name. This helps avoid attribute name collisions should subclasses inadvertently contain attributes with the same name.

>Note 1: Note that only the simple class name is used in the mangled name, so if a subclass chooses both the same class name and attribute name, you can still get name collisions.

>Note 2: Name mangling can make certain uses, such as debugging and __getattr__(), less convenient. However the name mangling algorithm is well documented and easy to perform manually.

>Note 3: Not everyone likes name mangling. Try to balance the need to avoid accidental name clashes with potential use by advanced callers.

# Dunder = magic methods

Pythonが予約してるメソッド

[ここ](https://dbader.org/blog/python-dunder-methods)

Over rideするといろいろできる

# Fucntions

In [73]:
def capitalize(text):
    print(text.upper() + '!')

capitalize('test1')

# プリントされる
test = capitalize
test('test2')

TEST1!
TEST2!


In [74]:
# 消したのでプリントされない
del capitalize
capitalize('retest1')


NameError: name 'capitalize' is not defined

In [75]:
# おおもとは消えたけど、代入されたのは残ってる
test('retest2')

RETEST2!


# function を function に渡す

In [79]:
def capitalize2(text):
    return text.upper() + '!'

def say(func):
    words = func('test test test')
    print(words)
    

say(capitalize2)

TEST TEST TEST!


# map の使い方

In [80]:
list(map(capitalize, ['aaa', 'bbb', 'ccc']))

['AAA!', 'BBB!', 'CCC!']

# nest function

In [94]:
# 呼べない
def say_something(context):
    def loud(context):
        return context.upper() + '!!!!!'

say = say_something('yo')
say.loud

AttributeError: 'NoneType' object has no attribute 'loud'

In [99]:
# 呼べる
def say_something(context):
    def loud(text):
        return text.upper() + '!!!!!'
    return loud(context)

say_something('yo')

'YO!!!!!'

## 場合分け

In [107]:
def say_something(context, loudness):
    def loud(text):
        return text.upper() + '!!!!!'
    def quietly(text):
        return text.lower() + '.......'

    if loudness == 'quietly':
        return quietly(context)
    if loudness == 'loud':
        return loud(context)

# 普通に呼ぶ
print(say_something('yo', 'loud'))
print(say_something('yo', 'quietly'))

YO!!!!!
yo.......


## child funcs have access to parent's local var

In [120]:
def say_something(text, loudness):
    def loud():
        return text.upper() + '!!!!!'
    def quietly():
        return text.lower() + '.......'

    if loudness == 'quietly':
        return quietly
    if loudness == 'loud':
        return loud

say_something('yo', 'quietly')()


'yo.......'

# closure

In [134]:
def tashizan(num):
    def tasu(x):
        return x + num
    return tasu

keisan = tashizan(1)
print(keisan)
print(keisan(2))

<function tashizan.<locals>.tasu at 0x7f39a8744840>
3


In [132]:
def print_out(text):
    return text
    
print(print_out)
print(print_out('hi'))

<function print_out at 0x7f39a9320f28>
hi


# Classもclosureっぽくできる

In [137]:
class TashiZan:
    def __init__(self, num):
        self.num = num
    
    def __call__(self, x):
        return self.num + x

tashizan = TashiZan(1)
print(tashizan)
print(tashizan(3))

4


# lambda

In [140]:
def my_tashizan(x, y):
    return x + y


print(my_tashizan(2, 3))

print((lambda x, y: x + y)(2, 3))

5
5


> Because return is a statement. Lambdas can only contain expressions.



In [144]:
tup = [(1, 'う'), (2, 'い'), (3, 'あ')]

sorted(tup, key=lambda x: x[1])

[(3, 'あ'), (2, 'い'), (1, 'う')]

# lambda で足し算

In [147]:
def tashizan(num):
    return lambda x: x + num

keisan = tashizan(1)
print(keisan)
print(keisan(2))

<function tashizan.<locals>.<lambda> at 0x7f39a8744730>
3


# decorator

In [190]:
def uppercase(func):
    print(func, '<= func')

    def wrapper():
        # funcでfunctionを渡して
        # カッコをつけて stringsにする
        print(func, '<= func()')
        original = func()
        return original.upper()

    # 最後に wrapper funcを返す
    print(wrapper, '<= wrapper')
    return wrapper


def uppercase2(strings):
    # say_hi() で渡すとstringが渡る
    return strings.upper()


@uppercase
def say_hi():
    return 'hi'

# これでもおなじこと
# print(uppercase2(say_hi()))

print(say_hi) # <= これだとwrapperが変えるだけ

#print(say_hi()) # <= これだとなかまで実行される


<function say_hi at 0x7f39a3d88a60> <= func
<function uppercase.<locals>.wrapper at 0x7f39a8087ae8> <= wrapper
<function uppercase.<locals>.wrapper at 0x7f39a8087ae8>


# multiple decorators

In [210]:
def moji_kazari1(func):
    print(func, 'func in moji_kazari1')
    def wrapper():
        return f"111111{func()}11111"
    print('exec wrapper in mojikazari1')
    return wrapper

def moji_kazari2(func):
    print(func, 'func in moji_kazari2')
    def wrapper():
        return f"22222{func()}22222"
    print('exec wrapper in mojikazari2')
    return wrapper

@moji_kazari2
@moji_kazari1
def say_smt():
    return 'yo'

say_smt()

<function say_smt at 0x7f39a3d88840> func in moji_kazari1
exec wrapper in mojikazari1
<function moji_kazari1.<locals>.wrapper at 0x7f39a3d88bf8> func in moji_kazari2
exec wrapper in mojikazari2


'22222111111yo1111122222'

# making debug-friendly decorator

## without functools wraps

所在がつかみにくい

In [219]:
def moji_kazari1(func):
    def wrapper():
        return f"111111{func()}11111"
    return wrapper

def moji_kazari2(func):
    def wrapper():
        return f"22222{func()}22222"
    return wrapper

@moji_kazari2
@moji_kazari1
def say_smt():
    """" just say yo... """
    return 'yo'

print(say_smt.__name__)
print(say_smt.__doc__)

wrapper
None


## with functools.wraps 

よくわかるようになった

In [218]:
import functools

def moji_kazari1(func):
    @functools.wraps(func)
    def wrapper():
        return f"111111{func()}11111"
    return wrapper

def moji_kazari2(func):
    @functools.wraps(func)
    def wrapper():
        return f"22222{func()}22222"
    return wrapper

@moji_kazari2
@moji_kazari1
def say_smt():
    """" just say yo... """
    return 'yo'

print(say_smt.__name__)
print(say_smt.__doc__)

say_smt
" just say yo... 


# args and kwargs

In [225]:
def args_test(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)
    
args_test('required1')
args_test('requried2', 'this', 'should', 'be', '*args')
args_test('requried3', 'this', 'should', 'be', '*args', {'this': 'will', 'be': 'kwargs'})
args_test('requried4', 'this', 'should', 'be', '*args', key1='this is key 1', key2='this is key2')
args_test('requried5', 'this', 'should', 'be', '*args', key1={'this': 'will', 'be': 'kwargs'})

required1
requried2
('this', 'should', 'be', '*args')
requried3
('this', 'should', 'be', '*args', {'this': 'will', 'be': 'kwargs'})
requried4
('this', 'should', 'be', '*args')
{'key1': 'this is key 1', 'key2': 'this is key2'}
requried5
('this', 'should', 'be', '*args')
{'key1': {'this': 'will', 'be': 'kwargs'}}


## args,  kwags and inheritance

In [250]:
class Car:
    def __init__(self, color, mileage, maker):
        self.color = color
        self.mileage = mileage
        self.maker = maker

class BlueCar(Car):
    def __init__(self, *args, **kwargs):
        print(args)
        print(kwargs)
        super().__init__(*args, **kwargs)
        self.color = 'blue'

BlueCar('red', 1234, 'Toyota').maker

('red', 1234, 'Toyota')
{}


'Toyota'

decoraterのwrapper, sub class などに args, kwargsとやっておくとarg が変わっても変更しなくていいから楽  
ただしどんなargが必要かわかりづらくなる

# \*\* and \* can unpack argument

In [271]:
def arg_test(x, y, z):
    return x, y, z


a_tuple = (1, 2, 3)
print(arg_test(*a_tuple))

a_dict = {'x': 1, 'y': 2, 'z': 3}
print(arg_test(**a_dict))

a_generater = (x * x for x in range(3))
print(a_generater)
print(arg_test(*a_generater))

(1, 2, 3)
(1, 2, 3)
<generator object <genexpr> at 0x7f39a3d2de60>
(0, 1, 4)


# is and ==

- "is" is for identity check
 - 「ID」が同じならTrue
- "==" is for equality check
 - 中身が「同じ」ならTrue

In [6]:
a = [1, 2, 3]
b = a

print(id(a))
print(id(b))
print(a is b)
print(a == b)

140617552321608
140617552321608
True
True


In [5]:
a = [1, 2, 3]
c = list(a)

print(id(a))
print(id(c))
print(a is c)
print(a == c)

140617564477512
140617552606280
False
True


# \_\_repr\_\_ and \_\_str\_\_

In [18]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

    def __str__(self):
        return f"str -- color: {self.color}, maileage: {self.mileage}--"

    def __repr__(self):
        return f"repr -- color: {self.color}, maileage: {self.mileage}--"


car = Car('red', '1234')
print(car)
car

str -- color: red, maileage: 1234--


repr -- color: red, maileage: 1234--

In [19]:
print(str(car))
print(repr(car))

str -- color: red, maileage: 1234--
repr -- color: red, maileage: 1234--


## repr str の使用例

In [22]:
import datetime
today = datetime.date.today()
print(str(today))
print(repr(today))

# repr はデバッグ用。str は表示用

2018-02-08
datetime.date(2018, 2, 8)


# exception class

## a bit vague

In [24]:
def validate_name(name):
    if len(name) > 6:
        raise ValueError

validate_name('aaaaaaaaa')

ValueError: 

## more explicit way

In [32]:
class BaseValidationError(ValueError):
    pass

class NameTooLongError(BaseValidationError):
    pass

class NameTooShortError(BaseValidationError):
    pass


def validate_name(name):
    if len(name) > 6:
        raise NameTooLongError(name)
    if len(name) < 3:
        raise NameTooShortError(name)

In [33]:
validate_name('aaaaaaaaa')

NameTooLongError: aaaaaaaaa

In [34]:
validate_name('aa')

NameTooShortError: aa

## shallow copy and deep copy

## shallow copy だと片方を変えるともう片方も変わる時が有る

In [41]:
list1 = [[0,1], [2,3]]
list2 = list(list1)

print(id(list1))
print(id(list2))

print('==============')

list1.append(['a list'])

print(list1)
print(list2)

print('==============')

list1[0][0] = 'changed!!'
print(list1)
print(list2)

140617550862664
140617550875336
[[0, 1], [2, 3], ['a list']]
[[0, 1], [2, 3]]
[['changed!!', 1], [2, 3], ['a list']]
[['changed!!', 1], [2, 3]]


## Deep copyすれば影響されない

In [44]:
import copy

list1 = [[0,1], [2,3]]
list2 = copy.deepcopy(list1)

print(id(list1))
print(id(list2))

print('==============')

list1.append(['a list'])

print(list1)
print(list2)

print('==============')

list1[0][0] = 'changed!!'
print(list1)
print(list2)

140617550863944
140617551356232
[[0, 1], [2, 3], ['a list']]
[[0, 1], [2, 3]]
[['changed!!', 1], [2, 3], ['a list']]
[[0, 1], [2, 3]]


# Object でも子オブジェクトに対して同じようなことが起きる

In [52]:
import copy

class NumGroup:
    def __init__(self, num_group):
        self.group = num_group

class Data:
    def __init__(self, data):
        self.data = data

data1 = Data(NumGroup([1,2,3]))
shallow_copied_data = copy.copy(data1)
deep_copied_data = copy.deepcopy(data1)

print('===変更するまえ===')
print(data1.data.group)
print(shallow_copied_data.data.group)
print(deep_copied_data.data.group)

print('===変更してみる===')
data1.data.group = [1000, 2000, 3000]
print(data1.data.group)
print(shallow_copied_data.data.group)
print(deep_copied_data.data.group)

===変更するまえ===
[1, 2, 3]
[1, 2, 3]
[1, 2, 3]
===変更してみる===
[1000, 2000, 3000]
[1000, 2000, 3000]
[1, 2, 3]


# Abstract Base Class

これを読む
https://docs.python.org/3.5/library/abc.html

## You can't instantiate a class without overriding all the method in Base class

In [84]:
from abc import ABCMeta, abstractmethod

class Base(metaclass=ABCMeta):
    @abstractmethod
    def method1(self):
        print('method1 from base')
    
    @abstractmethod
    def method2(self):
        print('method2 from base')

class Top(Base):
    def method1(self):
        pass

# You can't instantiate a class without overriding all the method in Base class
top = Top()
top.method1()

TypeError: Can't instantiate abstract class Top with abstract methods method2

## This is Okay

In [88]:
class Top2(Base):
    def method1(self):
        super().method1()
        print('medhod1 from Top2')

    def method2(self):
        super().method2()
        print('medhod2 from Top2')

# You can't instantiate a class without overriding all the method in Base class
top2 = Top2()
top2.method1()
print('============')
top2.method2()

method1 from base
medhod1 from Top2
method2 from base
medhod2 from Top2


In [71]:
from abc import ABCMeta, abstractmethod

class Animal(metaclass=ABCMeta):
    @abstractmethod
    def sound(self):
        pass

# 抽象クラスを継承しないが、 `sound`メソッドを実装
class Dog():
    pass

# 抽象クラスのAnimalへDogを登録
Animal.register(Dog)
Dog()

# collections.NamedTuple

In [107]:
from collections import namedtuple

Car = namedtuple('Car', 'color mileage')
my_car = Car('red', 1234)
print(my_car.color)
print(my_car.mileage)

# tupleなので値の変更はできない

# いろいろなヘルパー
print(my_car._asdict())
print(my_car._replace(color='blue')) # 直接変更しないでコピーを返す
print(Car._make(['black', 1234])) # for iterable

red
1234
OrderedDict([('color', 'red'), ('mileage', 1234)])
Car(color='blue', mileage=1234)
Car(color='black', mileage=1234)


# typing.NamedTuple - improved named tuple

In [120]:
from typing import NamedTuple

class Car(NamedTuple):
    color: str
    mileage: float
    automatic: bool

my_car = Car('red', 1234, True)
print(my_car)
print(my_car.color) 

# tuple なので書き換えはできない

Car(color='red', mileage=1234, automatic=True)
red


# types.SimpleNamespace

In [123]:
from types import SimpleNamespace

my_car = SimpleNamespace(color='red', mile=1234, stick=True)
print(my_car)
my_car.color = 'black'
print(my_car) # 書き換えできる

namespace(color='red', mile=1234, stick=True)
namespace(color='black', mile=1234, stick=True)


## subclassing

In [102]:
class CarColor(Car):
    def return_color(self):
        if self.color == 'red':
            return '赤'

a_car = CarColor('red', 1234)
print(a_car.return_color())

赤


# instance method, class method, static method

In [21]:
class Test:

    # self を arg にとる
    # self.__class__ でclassにもアクセスできる
    def instance_method(self):
        return 'this is instance method', self
    
    # class しか読めないので、self はいじれない
    @classmethod
    def classmethod(cls):
        return 'this is class method', cls
    
    # selfもclassもargにないので値を変更できない
    @staticmethod
    def staticmethod():
        return 'this is static method'

test = Test()

# instance method どちらでも良い。
# self をarg にしてるので、selfが返る
print(test.instance_method())

# class methodはclassがargになるので、class が返る
print(test.classmethod())

# statice method はなにもargにしてないので何も帰らない
print(test.staticmethod())

print(Test.classmethod())
print(Test.staticmethod())
print(Test.instance_method(test)) # Test.instance_method() ではエラー


('this is instance method', <__main__.Test object at 0x7f1effd89b70>)
('this is class method', <class '__main__.Test'>)
this is static method
('this is class method', <class '__main__.Test'>)
this is static method
('this is instance method', <__main__.Test object at 0x7f1effd89b70>)


# 使い方

- class method ->  factory として使う。 __init_は一個しか使えないので。
- interface として使う。値の変更ができないので

In [69]:
class Jirou:
    def __init__(self, mashi_num, toppings):
        self.mashi_num = mashi_num
        self.toppings = toppings

    def __repr__(self):
        return f'ラーメン {self.toppings}{self.topping_mashi()}'

    @classmethod
    def dai_jirou(cls):
        return cls(0, ['大二郎'])

    @classmethod
    def zen_mashi(cls):
        return cls(0, ['ヤサイ', 'ニンニク', 'アブラ', 'カラメ'])
    
    def topping_mashi(self):
        return self.mashi(self.mashi_num)
    
    @staticmethod
    def mashi(mashi_num):
        return 'マシ' * mashi_num


print(Jirou(4, ['ニンニク']))
print(Jirou.dai_jirou())
print(Jirou.zen_mashi())

order = Jirou(2, ['ニンニク'],  )
print(order.topping_mashi())
print(Jirou.mashi(2))

ラーメン ['ニンニク']マシマシマシマシ
ラーメン ['大二郎']
ラーメン ['ヤサイ', 'ニンニク', 'アブラ', 'カラメ']
マシマシ
マシマシ


# ordered dict - 順番を維持

In [89]:
import collections

d1 = {'one': 1, 'two': 2, 'three': 3}
d2 = collections.OrderedDict(one=1, two=2, three=3)

print(d1)
print(d2)

print(d1.keys())
print(d2.keys())

{'one': 1, 'two': 2, 'three': 3}
OrderedDict([('one', 1), ('two', 2), ('three', 3)])
dict_keys(['one', 'two', 'three'])
odict_keys(['one', 'two', 'three'])


# default dict - キーがないときは callable を実行する

In [106]:
from collections import defaultdict

# キーがないときは callable を実行する
dd = defaultdict(list)
dd['greetings'].append('yo')
dd['greetings'].append('yoyo')
dd['greetings'].append('yoyoyo')
dd

defaultdict(list, {'greetings': ['yo', 'yoyo', 'yoyoyo']})

# chain map - dict を zip

In [112]:
from collections import ChainMap

dict1 = {'a': 1, 'b': 2}
dict2 = {'c': 3, 'd': 4}

chain = ChainMap(dict1, dict2)
print(chain)

# 左から右に探していく
print(chain['a'])


ChainMap({'a': 1, 'b': 2}, {'c': 3, 'd': 4})
1


# Mapping Proxy Type - dict を readonlyに

In [115]:
from types import MappingProxyType

writable = {'one': 1 , 'two': 2}
read_only = MappingProxyType(writable)

# 変更を受け付けない
read_only['one'] = 111

TypeError: 'mappingproxy' object does not support item assignment

In [116]:
# 食わせた方なら受け付ける
writable['two'] = 111
read_only

mappingproxy({'one': 1, 'two': 111})

# array

- str も array
- array.array はひとつのtype しか受け付けないがパフォーマンスが良い, numpyとかpandasとかいろいろある

# Queue and Stack

## Stack(LIFO)

- list 
  - FIROで右側から突っ込んで取り出すだけならO(1)で効率が良い
  - LIFOだとO(n)になるのでダメ

- collections.deque
 - O(1)でいける

In [5]:
from queue import LifoQueue

a_queue = LifoQueue()

a_queue.put(1)
a_queue.put(2)
a_queue.put(3)

print(a_queue)

# 尻からしか取り出せない
print(a_queue.get())
print(a_queue.get())
print(a_queue.get())

3
2
1


## Queue(FIFO)

- collections.deque
  - アタマでも尻でもO(1)
- queue.Queue, multiprocessing.Queue
  - 並列、並行処理向け

# loops

In [20]:
my_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
my_dict = {'a': 1, 'b': 2, 'c': 3}

for letter in my_list:
    print(letter)

print("=================")

for ind, letter in enumerate(my_list):
    print(f"{ind}: {letter}")

print("=================")

print(my_dict.items())

for key, value in my_dict.items():
    print(f"{key}: {value}")


print("===2飛び=====")

for num in range(0, len(my_list)-1, 2):
    print(my_list[num])

a
b
c
d
e
f
g
0: a
1: b
2: c
3: d
4: e
5: f
6: g
dict_items([('a', 1), ('b', 2), ('c', 3)])
a: 1
b: 2
c: 3
===2飛び=====
a
c
e


# Slicing

- list[start:end:step]

# \_\_iter\_\_ and  \_\_next\_\_

In [5]:
class Repeater:
    def __init__(self, value, max_value):
        self.value= value
        self.max_value = max_value
        self.count = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.count += 1
        if self.count > self.max_value:
            # これがないと止まらない
            raise StopIteration
        return self.value + ("ー" * self.count)

rep = Repeater('@', 5)

for item in rep:
    print(item)



@ー
@ーー
@ーーー
@ーーーー
@ーーーーー


# Generator

## 上と同じことをしてみる

In [8]:
def repeater(value, max_value):
    count = 0
    while True:
        count += 1
        if count > max_value:
            return
        yield value + ('ー' * count)


for word in repeater('＠', 5):
    print(word)

＠ー
＠ーー
＠ーーー
＠ーーーー
＠ーーーーー


## Print するときにList全生成するので効率が悪い(数万とか無理)

In [12]:
for num in list(range(0, 5)):
    print(num)

0
1
2
3
4


## Generator を使うとその都度生成するので楽

In [23]:
def generate_list(max_num):
    num = 0
    while True:
        if num >= max_num:
            return
        yield num
        num += 1

for num in generate_list(5):
    print(num)

0
1
2
3
4


## Generator Expression (Syntax Sugar)

In [25]:
iterator = ('yo' for i in range(3))

print(iterator)
print(iterator.__next__())
print(iterator.__next__())
print(iterator.__next__())
print(iterator.__next__())

<generator object <genexpr> at 0x7f4da723da98>
yo
yo
yo


StopIteration: 

## Generator Expression (Syntax Sugar)

In [24]:
for num in (num for num in range(5)):
    print(num)

0
1
2
3
4


# Generator Chain

In [40]:
integers = range(8)
print(integers)

squared = (i * i for i in integers)

# generater が返る
print(squared)

# 何かでつかう
negated = (-i for i in squared)
print(negated)

print(list(negated))

# 一回イテレーションされたので累乗の方は消える
print(list(squared))

range(0, 8)
<generator object <genexpr> at 0x7f4da723d990>
<generator object <genexpr> at 0x7f4da723d678>
[0, -1, -4, -9, -16, -25, -36, -49]
[]


# Dictionary の使い方

In [49]:
my_dict = {'1': 'a', '2': 'b', '3': 'c'}


# 二回Dictを触るので効率悪い
def get_value(num):
    if num in my_dict:
        return my_dict[num]
    else:
        return 'Not Found'

print(get_value(11111))


# まだまし
def get_value2(num):
    try:
        return my_dict[num]
    except KeyError:
        return 'Not Found'

print(get_value(11111))

# もっといい方法
def get_value3(num):
    return my_dict.get(11111, 'Not Found')

print(get_value(11111))


Not Found
Not Found
Not Found


# Dict はキーのHashが同じで、equal true だと同じものとみなされる

In [60]:
print({True: 'yes', 1: 'no', 1.0: 'maybe'})
print(True == 1 == 1.0)

{True: 'maybe'}
True


In [59]:
class SameHash:
    def __hash__(self):
        return 1


a = SameHash()
b = SameHash()

print({a: 'a', b: 'b'})


class AlwaysEqual:
    def __eq__(self, other):
        return True

    def __hash__(self):
        return id(self)


a = AlwaysEqual()
b = AlwaysEqual()

print({a: 'a', b: 'b'})


class HashEqEqual:
    def __eq__(self, other):
        return True

    def __hash__(self):
        return 1


a = HashEqEqual()
b = HashEqEqual()

print({a: 'a', b: 'b'})

{<__main__.SameHash object at 0x7f4da78bdeb8>: 'a', <__main__.SameHash object at 0x7f4da78bdf28>: 'b'}
{<__main__.AlwaysEqual object at 0x7f4da77b7668>: 'a', <__main__.AlwaysEqual object at 0x7f4da77c5390>: 'b'}
{<__main__.HashEqEqual object at 0x7f4da78bdef0>: 'b'}


>辞書においては、値の設定、取得の際に、__eq__ による全体の探索を避けて、低コストで処理したいという要求に応えるためハッシュ関数が利用されています。

>値の設定時に上書きするか追加するかを素早く判断
ハッシュ値が一致しかつ __eq__ で一致するものものがない→新しいキーを追加
ハッシュ値が一致しかつ __eq__ で一致するものがある→既存のキーの値を上書き
値の取得時に一致するものを素早く見付ける
ハッシュ値が一致しかつ __eq__ で一致するものがない→エラー
ハッシュ値が一致しかつ __eq__ で一致するものがある→値を返す